## Reconstruct from original text
> this file contains every field from the azure transformations,
> you can restart from here if you want to construct new features

In [1]:
pwd

'/home/ucsddse230/work/final_project/post_azure'

In [2]:
import pyspark.ml.feature 
from pyspark.ml.feature import CountVectorizer
import pyspark.sql.functions  as psf
from pyspark.sql import Window 
from pyspark.sql.types  import DateType
import pandas as pd
#pd.set_option('display.max_colwidth', -1)
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType

In [3]:
#sc.stop()
import findspark
findspark.init()
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType , DateType, ArrayType
from pyspark.sql import SparkSession
from pyspark import SparkContext ,SparkConf
SparkContext.setSystemProperty('spark.executor.memory', '6g')
#SparkContext.setSystemProperty('spark.rdd.compress', True)
SparkContext.setSystemProperty('spark.driver.memory', '16g')
sc = SparkContext(master="local[4]")
print(sc)


<SparkContext master=local[4] appName=pyspark-shell>


In [4]:
path='/home/ucsddse230/work/final_project/azure/Tokens_vectors_text/final_tokens_with_text.parquet'

In [5]:
spark = SparkSession.builder.getOrCreate()

### this file contains everything that happend before vectorization
#### includung ** original text **

In [6]:
processed_azure_token_original_text= spark.read.parquet(path)

In [7]:
processed_azure_token_original_text.dtypes

[('HADM_ID', 'int'),
 ('SUBJECT_ID', 'int'),
 ('ADMITTIME', 'timestamp'),
 ('DISCHTIME', 'timestamp'),
 ('DEATHTIME', 'timestamp'),
 ('ADMISSION_TYPE', 'string'),
 ('NEXT_ADMISSION', 'timestamp'),
 ('NEXT_ADMISSION_TYPE', 'string'),
 ('days_next_admit', 'int'),
 ('CATEGORY', 'string'),
 ('TEXT', 'string'),
 ('row_number', 'int'),
 ('label', 'int'),
 ('TEXT_clean', 'string'),
 ('tokens', 'array<string>'),
 ('tokens_clean', 'array<string>')]

In [8]:
 processed_azure_token_original_text[["tokens"]].take(1)

[Row(tokens=['admission', 'date', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'discharge', 'date', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'date', 'of', 'birth', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'sex', '', '', 'm', '', 'service', '', '', 'acove', '', 'history', 'of', 'present', 'illness', '', '', '', 'this', 'is', 'a', '', '', '', 'year', 'old', 'male', 'with', 'hiv', 'who', 'presented', 'on', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'with', 'intermittent', 'fevers', '', 'chills', 'and', 'diaphoresis', 'times', 'five', 'weeks', '', 'progressive', 'shortness', 'of', 'breath', 'and', 'cough', 'times', 'two', 'to', 'three', 'weeks', '', '', 'the', 'cough', 'was', 'productive', 'of', 'white', 'sputum', '', '', 'the', 'patient', 'also', 'reports', 'a', 'sore', 'throat', '', 'nausea', 'and', 'vomiting', 'times', 'one

## prepare Training test and validation data

In [9]:
#grab relavant fields
subset_for_features=processed_azure_token_original_text.select('HADM_ID', 'SUBJECT_ID', 'label','tokens_clean' )

In [10]:
seed=6250

In [11]:
splits=subset_for_features.randomSplit([.8, 0.2], seed)

In [12]:
print("Training obs {} , testing obs {}".format(splits[0].count(),splits[1].count()))

Training obs 40795 , testing obs 10318


In [13]:
#extract training testing and validation data
training=splits[0]
testing=splits[1]

In [14]:
token_vector = CountVectorizer(inputCol="tokens_clean", outputCol="features")
vector_model=token_vector.fit(training)
# .transform(final_data).cache()

### Model Terms

In [15]:
vector_model.vocabulary

['patient',
 'blood',
 'discharge',
 'daily',
 'sig',
 'one',
 'left',
 'history',
 'right',
 'pain',
 'date',
 'normal',
 'chest',
 'well',
 'home',
 'status',
 'capsule',
 'stable',
 'follow',
 'course',
 'disease',
 'care',
 'days',
 'two',
 'artery',
 'aortic',
 'wbc',
 'valve',
 'neg',
 'medical',
 'negative',
 'glucose',
 'past',
 'per',
 'post',
 'cardiac',
 'exam',
 'acute',
 'mild',
 'present',
 'pulmonary',
 'family',
 'ventricular',
 'heart',
 'physical',
 'without',
 'edema',
 'rbc',
 'weeks',
 'hgb',
 'procedure',
 'surgical',
 'due',
 'urine',
 'surgery',
 'pressure',
 'condition',
 'evidence',
 'mch',
 'small',
 'renal',
 'delayed',
 'lower',
 'mitral',
 'non',
 'hypertension',
 'systolic',
 'chronic',
 'tube',
 'coronary',
 'fluid',
 'failure',
 'soft',
 'head',
 'abdominal',
 'illness',
 'aspirin',
 'social',
 'atrial',
 'calcium',
 'rate',
 'old',
 'abdomen',
 'effusion',
 'bowel',
 'respiratory',
 'changes',
 'wall',
 'secondary',
 'low',
 'bilateral',
 'floor',
 'th

In [16]:
training_features = vector_model.transform(training).cache()
testing_features = vector_model.transform(testing).cache()

In [21]:
#notice the same length features
print('------------------Training----------------------------')
print(training_features[["features"]].take(1))
print('------------------------------------------------------')
print('--------------Testing---------------------------------')
print('------------------------------------------------------')
print(testing_features[["features"]].take(1))

------------------Training----------------------------
[Row(features=SparseVector(4499, {0: 5.0, 1: 2.0, 2: 7.0, 3: 1.0, 4: 5.0, 5: 4.0, 6: 8.0, 7: 6.0, 8: 1.0, 9: 13.0, 10: 3.0, 11: 1.0, 12: 1.0, 13: 2.0, 14: 2.0, 15: 1.0, 16: 3.0, 17: 2.0, 18: 4.0, 19: 2.0, 21: 1.0, 23: 1.0, 26: 1.0, 29: 1.0, 31: 1.0, 32: 2.0, 34: 1.0, 36: 2.0, 39: 1.0, 41: 2.0, 44: 1.0, 45: 1.0, 47: 1.0, 48: 1.0, 49: 1.0, 50: 1.0, 51: 2.0, 52: 1.0, 53: 1.0, 54: 1.0, 56: 2.0, 57: 1.0, 58: 1.0, 60: 4.0, 62: 1.0, 68: 1.0, 72: 1.0, 75: 1.0, 76: 1.0, 77: 1.0, 81: 1.0, 85: 1.0, 91: 1.0, 98: 1.0, 99: 1.0, 100: 1.0, 102: 1.0, 104: 1.0, 107: 1.0, 109: 1.0, 113: 1.0, 118: 1.0, 119: 1.0, 123: 2.0, 132: 1.0, 143: 2.0, 144: 3.0, 145: 1.0, 164: 1.0, 167: 2.0, 171: 1.0, 178: 1.0, 181: 1.0, 184: 2.0, 189: 1.0, 190: 1.0, 191: 1.0, 195: 2.0, 196: 2.0, 198: 1.0, 207: 1.0, 209: 1.0, 213: 1.0, 222: 1.0, 230: 1.0, 233: 1.0, 241: 2.0, 244: 1.0, 245: 1.0, 249: 1.0, 257: 1.0, 275: 1.0, 280: 1.0, 302: 2.0, 327: 1.0, 342: 2.0, 344: 1.0, 369: 

In [18]:
pwd

'/home/ucsddse230/work/final_project/post_azure'

### save data sets as parquet

In [20]:
training_features.write.parquet("training_features.parquet")
testing_features.write.parquet("testing_features.parquet")